In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time 
import pandas as pd
import json
import requests
from pyquery import PyQuery
import numpy as np
from requests.adapters import HTTPAdapter, Retry

In [2]:
def get_links(page_num)
    # Extraction of all links till june 2020
    service = Service(ChromeDriverManager().install())


    #driver = webdriver.Chrome(service=service)
    browser = webdriver.Chrome(service=service)
    urls = []
    articles_link = ["Links"]
    # since the tops category has 7 pages, where link to each follows a specific pattern identified above, we can create links to them as following:
    urls.append('https://hindi.thequint.com/news/webqoof')

    for i in range(1, page_num):
        urls.append('https://hindi.thequint.com/news/webqoof/' + str(i))


    # extracting links for products in each page
    for url in urls:
        # open the url
        browser.get(url)
        # purposeful wait time to allow the website to get fully loaded
        time.sleep(4)
        # get page content
        content = browser.page_source
        soup = BeautifulSoup(content, "lxml")
        art_links = []
        # extract all the anchor i.e., <a> elements with “thumb-link” class from the page
        #data_links = soup.find_all("a", {"class": "headline-link _3aBL6"})

        #data1_links = soup.find_all("a", {"class": "title-wrapper headline-link _3b9Yy"})

        data2_links = soup.find_all("a", {"class": "headline-link"})
        # from each <a> element, extract the URL

        for i in data2_links:
            art_links.append(i['href'])

        articles_link.extend(art_links[10:])
        # purposeful wait time to avoid sending requests in quick succession
        time.sleep(10)
    browser.quit()
    np.savetxt("Quint_webqoof_hindi_links_june2020.csv", articles_link, delimiter =", ", fmt ='% s')
    return

In [2]:
def get_content(dt):
    for url in dt["Links"]:
        #url = "https://fit.thequint.com/fit-webqoof/drinking-milk-after-eating-fish-dangerous-fact-check"
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)

        response = session.get(url)
        # Check if the request was successful
        if response.status_code == 200:
            linnnks.append(url)
            # Parse the HTML content of the page using BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tags = soup.find_all('meta')
            meta_features = {}
            for tag in meta_tags:
                if tag.get('property'):
                    meta_features[tag['property']] = tag.get('content')
            # title
            title.append(meta_features['og:title'])

            # Published_date
            if 'article:published_time' in meta_features:
                Published_date.append(meta_features['article:published_time'])
            elif 'article:published' in meta_features:
                Published_date.append(meta_features['article:published'])
            else:
                Published_date.append(" ")
            # top image
            top_image.append(meta_features['og:image'])

            # Content
            data = soup.find_all('div', {'class':"story-element story-element-text"})
            if data:
                extracted_content = []
                investigation_content = ""
                booll = 0
                for element in data:
                    for el in element.find_all(['p', 'blockquote']):
                        extracted_content.append(el.get_text())
                        if booll == 0:
                            claim.append(el.get_text())
                            booll = 1
                        else:
                            investigation_content += el.get_text() + "\n"

                full_content = '\n'.join(extracted_content)

            Content.append(full_content)
            investigation.append(investigation_content) 

            # Extract all links
            links = soup.select('div.story-element-text a')
            not_include = ["https://www.thequint.com/subscribe", "https://t.me/TheQuint", "https://www.thequint.com/news/webqoof"]
            # Extract and print the links
            linksss = []
            for link in links:
                href = link.get('href')
                if href not in not_include:
                    if link:
                        linksss.append(href)

            links_in_text.append(linksss)
    print(len(title))
    print(len(Published_date))
    print(len(Content))
    print(len(claim))
    print(len(investigation))
    print(len(top_image))
    print(len(links_in_text))
    print(len(linnnks) )   
    #save to csv file 
    df1 = pd.DataFrame({'link':linnnks, 'title': title, 'publish_date': Published_date, 'content': Content, 'investigation': investigation, 'Links_in_text':links_in_text,'top image': top_image})
    df1.to_csv("Quint_english_june2020.csv")
    return

In [3]:
if __name__ == "__main__":
    title = []
    Published_date = []
    Content = []
    claim = []
    investigation = []
    top_image = []
    links_in_text = []
    linnnks = []
    get_links(200)
    dataframe = pd.read_csv("Quint_webqoof_english_links_june2020.csv")
    get_content(dataframe)

3091
3091
3091
3084
3091
3091
3091
3091
